<a href="https://colab.research.google.com/github/ellabam/learning-java/blob/master/Copy_of_Exercise_Text_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np  #load up the libraries and object defs. we need
import pandas as pd
from pandas import DataFrame, Series

# load up my visualization system, and call the object plt
import matplotlib.pyplot as plt

# tell ipython notebook to print visualizations inline
%pylab
%matplotlib inline

# NLTK

Import the nltk (Natural Language Toolkit) package.

In [ ]:
import nltk

NTLK doesn't contain many language packages by default, so you'll need to install a few. If you haven't installed the `punkt` tokenizer package, you'll need to uncomment and run the line below to launch the NLTK downloader. 

Once it's loaded, enter `d` and then download the `punkt` package.

While you're there. Also download the `averaged_perceptron_tagger`, `stopwords`, and `vader_lexicon` packages. (We'll use them later.)

In [ ]:
nltk.download_shell()

## Text Splitting Basics

In [ ]:
s = "This is a short, simple piece of text, so let's try a short, simple analysis."

Breaking it up with `string.split()`

In [ ]:
s.lower()

In [ ]:
s.split()

## Tokenizing
Using NLTK, we can split ("tokenize") more intelligently.

In [ ]:
nltk.wordpunct_tokenize(s)

We can also count and visualize the distribution of terms.

In [ ]:
tokens = nltk.wordpunct_tokenize(s)
nltk.probability.FreqDist(tokens)

In [ ]:
fd = nltk.probability.FreqDist(tokens)
fd.plot()

### Removing stop words
NLTK provides corpuses of common stop words that we can use to remove common terms from text.

In [ ]:
stopwords = nltk.corpus.stopwords.words('english')

Here's a slow way to remove stop words using a for loop.

In [ ]:
t = s.lower()   # make lowercase
tokens = nltk.tokenize.wordpunct_tokenize(t)
no_stopwords = []  # this will hold our NON-stopword words
for t in tokens:   # for each word
    if t not in stopwords:  # not in stopword list
        no_stopwords.append(t)  # collect the word

print(no_stopwords)

#### A nice Python shortcut
You can actually do this kind of transformation more concisely by using list comprehensions.

In [ ]:
tokens = nltk.wordpunct_tokenize(s)
print(tokens)   # non-lowercase version

In [ ]:
lower_tokens = [t.lower() for t in tokens]

# ^ this list comprenehsion is basically a shorthand for:
#lower_tokens = []
#for t in tokens:
#    lower_tokens.append(t.lower())
print(lower_tokens)

So to remove stopwords, we can nest an if statement inside the list comprehension.

In [ ]:
tokens_nostop = [t.lower() for t in tokens if t not in stopwords]
print(tokens_nostop)

You can remove punctuation the same way.

In [ ]:
import string
string.punctuation

In [ ]:
tokens_nopunct = [t.lower() for t in tokens if t not in string.punctuation]
print(tokens_nopunct)

Or we can do both.

In [ ]:
tokens_nostop_nopunct = [t.lower() for t in tokens if (t not in string.punctuation and t not in stopwords)]
print(tokens_nostop_nopunct)

### Stemming

In [ ]:
stemmer = nltk.stem.snowball.EnglishStemmer()  # grab me a "snowball stemmer"

In [ ]:
stemmer.stem("runs")

In [ ]:
stemmer.stem("applicable")

Lowercase, remove stopwords, and stem all in one line.

In [ ]:
filtered = [stemmer.stem(t.lower()) for t in tokens if (t not in string.punctuation and t.lower() not in stopwords)]
print(filtered)

In [ ]:
print(s)
nltk.probability.FreqDist(filtered).plot()

### Parts of Speech
Labels each term in the text with its part of speech (noun, verb, adjective, etc.). 

Parts of speech are labeled using the codes from the UPENN Treebank project: https://www.ling.upenn.edu/courses/Fall_2003/ling001/penn_treebank_pos.html

In [ ]:
nltk.pos_tag(tokens_nopunct)

In [ ]:
pos_list = [i[1] for i in nltk.pos_tag(tokens_nopunct)]
nltk.probability.FreqDist(pos_list).plot()

### n-grams

In [ ]:
n = 3
[i for i in nltk.ngrams(tokens_nopunct,n)]

### Sentiment
Sentiment analysis can be interesting, but it's also **very** inexact. You should always take it with a grain of salt, especially on small corpuses of text.

In [ ]:
from nltk.sentiment.vader import SentimentIntensityAnalyzer
sid = SentimentIntensityAnalyzer()

In [ ]:
print(s)
sid.polarity_scores(s)

^ The compound score here combines the others and will for negative sentences/phrases deemed to have negative sentiment and positive for text deemed to have positive sentiment.

In [ ]:
sid.polarity_scores("The movie was atrocious with horrible acting and a deeply flawed plot.")

In [ ]:
sid.polarity_scores("The movie was amazing with stellar acting and an incredible plot.")

## Tweet data
Let's try this on a small sample of ~300 tweets from [@BoredElonMusk](https://twitter.com/BoredElonMusk?ref_src=twsrc%5Egoogle%7Ctwcamp%5Eserp%7Ctwgr%5Eauthor).

In [ ]:
df = pd.read_csv("https://drive.google.com/uc?export=download&id=15J4CzqPEVVkPbaFUSko288_ppcXJG4hE")

In [ ]:
df.head(10)

### Tweet length

In [ ]:
df['tweet-length'] = [len(t) for t in df['tweet-text']]
print(df.head(5))
df['tweet-length'].hist()

### Sentiment

In [ ]:
df['sentiment'] = df['tweet-text'].map(lambda x: sid.polarity_scores(x)["compound"])
print(df[['tweet-text','sentiment']].head(5))
df['sentiment'].hist()

### Word frequency
Tokenize the tweet text and remove punctuation and stopwords.

In [ ]:
df['tweet-tokens'] = df['tweet-text'].map(lambda x: nltk.wordpunct_tokenize(x))
df[['tweet-text','tweet-tokens']].head(5)

In [ ]:
def remove_stop_punct(token_list):
    return [t.lower() for t in token_list if (t not in string.punctuation and t.lower() not in stopwords)]

df['tweet-tokens'] = [nltk.wordpunct_tokenize(tweet) for tweet in df['tweet-text']]
df['tweet-tokens-filtered'] = df['tweet-tokens'].map(remove_stop_punct)
df.head(5)


Get a list of all of the tokens in the whole corpus

In [ ]:
tweet_tokens_filtered_list = [item for sublist in df["tweet-tokens-filtered"] for item in sublist]
nltk.FreqDist(tweet_tokens_filtered_list).plot(30) #Show me just the top 30

# ToDo
Are there terms that should be removed from this frequency distribution? Think about how you'd remove them.

*Hint: Think about how we removed punctuation and stop words earlier.*